# Implementing backpropagation

## Data Cleanup

In [46]:
import numpy as np
import pandas as pd

### Load the Data

In [47]:
admissions = pd.read_csv('A4_binary.csv')
admissions[:10]

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
5,1,760,3.00,2
6,1,560,2.98,1
7,0,400,3.08,2
8,1,540,3.39,3
9,0,700,3.92,2


### One-hot encoding

In [48]:
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data[:10]

,admit,gre,gpa,rank,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,3,0,0,1,0
1,1,660,3.67,3,0,0,1,0
2,1,800,4.00,1,1,0,0,0
3,1,640,3.19,4,0,0,0,1
4,0,520,2.93,4,0,0,0,1
5,1,760,3.00,2,0,1,0,0
6,1,560,2.98,1,1,0,0,0
7,0,400,3.08,2,0,1,0,0
8,1,540,3.39,3,0,0,1,0
9,0,700,3.92,2,0,1,0,0


In [49]:
data = data.drop('rank', axis=1)
data[:10]

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
5,1,760,3.00,0,1,0,0
6,1,560,2.98,1,0,0,0
7,0,400,3.08,0,1,0,0
8,1,540,3.39,0,0,1,0
9,0,700,3.92,0,1,0,0


### Scale the data

In [50]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std 
data[:10]

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
5,1,1.491561,-1.024525,0,1,0,0
6,1,-0.239793,-1.077078,1,0,0,0
7,0,-1.624876,-0.814312,0,1,0,0
8,1,-0.412928,0.000263,0,0,1,0
9,0,0.972155,1.392922,0,1,0,0


### Split off random 10% of the data for testing

In [51]:
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample) # .ix: indexing using both position and label

In [52]:
print(data[:10])

     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
106      1  0.972155  0.446965       1       0       0       0
9        0  0.972155  1.392922       0       1       0       0
61       0 -0.239793 -0.183673       0       0       0       1
224      0  1.837832 -1.287291       0       1       0       0
37       0 -0.586063 -1.287291       0       0       1       0
242      1  0.799020 -1.129631       0       0       1       0
313      1 -0.586063  0.683454       0       0       0       1
52       0  1.318426 -0.052290       0       0       0       1
347      0 -0.066657  1.077603       0       1       0       0
239      0  0.972155  0.157922       0       0       1       0


In [53]:
print(test_data[:10])

     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
48       0 -1.278605 -2.390908       0       0       0       1
50       0  0.452749  1.235263       0       0       1       0
80       0  0.972155 -1.287291       0       0       0       1
84       1 -0.759199  0.552071       0       0       1       0
98       0  0.972155 -1.339844       0       1       0       0
110      0  0.799020 -0.814312       0       0       0       1
120      1 -0.586063  0.919944       0       1       0       0
122      0 -0.586063 -1.418674       0       0       1       0
133      0 -0.759199 -0.814312       0       0       1       0
148      1 -0.932334 -1.261014       1       0       0       0


### Split into features and targets

In [54]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']

In [55]:
print(features[:10])

          gre       gpa  rank_1  rank_2  rank_3  rank_4
106  0.972155  0.446965       1       0       0       0
9    0.972155  1.392922       0       1       0       0
61  -0.239793 -0.183673       0       0       0       1
224  1.837832 -1.287291       0       1       0       0
37  -0.586063 -1.287291       0       0       1       0
242  0.799020 -1.129631       0       0       1       0
313 -0.586063  0.683454       0       0       0       1
52   1.318426 -0.052290       0       0       0       1
347 -0.066657  1.077603       0       1       0       0
239  0.972155  0.157922       0       0       1       0


In [56]:
print(targets[:10])

106    1
9      0
61     0
224    0
37     0
242    1
313    1
52     0
347    0
239    0
Name: admit, dtype: int64


In [57]:
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [58]:
print(features_test[:10])

          gre       gpa  rank_1  rank_2  rank_3  rank_4
48  -1.278605 -2.390908       0       0       0       1
50   0.452749  1.235263       0       0       1       0
80   0.972155 -1.287291       0       0       0       1
84  -0.759199  0.552071       0       0       1       0
98   0.972155 -1.339844       0       1       0       0
110  0.799020 -0.814312       0       0       0       1
120 -0.586063  0.919944       0       1       0       0
122 -0.586063 -1.418674       0       0       1       0
133 -0.759199 -0.814312       0       0       1       0
148 -0.932334 -1.261014       1       0       0       0


In [59]:
print(targets_test[:10])

48     0
50     0
80     0
84     1
98     0
110    0
120    1
122    0
133    0
148    1
Name: admit, dtype: int64


## Backpropagation

In [60]:
import numpy as np
# from data_prep import features, targets, features_test, targets_test

In [61]:
np.random.seed(21)

In [62]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [63]:
# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

In [64]:
n_records, n_features = features.shape
last_loss = None

print(n_records)
print(n_features)
print(last_loss)

360
6
None


In [65]:
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [66]:
print(weights_input_hidden)

[[-0.02121432 -0.0453956 ]
 [ 0.42531176 -0.51306167]
 [ 0.30430325 -0.69853477]
 [-0.08404378 -0.09576333]
 [ 0.46056288 -0.00515452]
 [-0.25033797  0.56080598]]


In [67]:
print(weights_hidden_output)

[ 0.65768472 -0.28137626]


In [72]:
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        
        output = sigmoid(np.dot(hidden_output, weights_hidden_output)
)
        ## Alternatively,
        # output_layer_in = np.dot(hidden_output, weights_hidden_output)
        # output = sigmoid(output_layer_in)

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y - output

        # TODO: Calculate error term for the output unit
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        
        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        
        # TODO: Update the change in weights
        del_w_hidden_output +=  output_error_term * hidden_output
        del_w_input_hidden +=  hidden_error_term * x[:, None]

    # TODO: Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.23162657246104928
Train loss:  0.2316263664902962
Train loss:  0.23162616052481885
Train loss:  0.23162595456464072
Train loss:  0.23162574860979196
Train loss:  0.23162554266029245
Train loss:  0.23162533671616992
Train loss:  0.2316251307774499
Train loss:  0.2316249248441594
Train loss:  0.23162471891632047


In [69]:
# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725
